# 1. Specify all parameters to be varied


In [1]:
from fedt import *

CAD_variables = [{
        'name' : 'radius',
        'data type' : 'ratio',
        'test_values' : [x for x in range(10, 20, 5)],
        'argname' : 'radius'
    }]
CAM_variables = [{
        'name' : 'cut power',
        'data type' : 'ratio',
        'test_values' : [x for x in range(90, 100, 5)],
        'argname' : 'cut power'
    },
    {
        'name' : 'cut speed',
        'data type' : 'ratio',
        'test_values' : [x for x in range(5, 100, 25)],
        'argname' : 'cut speed'
    }]

fab_repetitions = 2

sample_post_process_variable = {
    'name' : 'paint type',
    'data type' : 'nominal',
    'test_values' : ['lacquer', 'spraypaint'],
    'description' : 'use the {} on the object'
}
post_process_variables = [sample_post_process_variable]
post_process_repetitions = 1

sample_interaction_variable = {
    'name' : 'user push force level',
    'data type' : 'ratio',
    'test_values' : [x for x in range(10)],
    'instruction' : 'push on the button with force {} / 10' 
}
interaction_variables = [sample_interaction_variable]

measurement_variables = [{
        'name' : 'printed x-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'printed y-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'printed z-axis',
        'data type' : 'ratio'
    }
]
measurement_repetitions = 5

tea_hypothesis = (['material', 'measured z-axis' ], ['fill_pattern:concentric > triangle'])

###########

experiment_size(CAD_variables, CAM_variables, fab_repetitions,
                post_process_variables, post_process_repetitions,
                interaction_variables,
                measurement_variables, measurement_repetitions)

vars_to_labels = label_all_conditions(CAD_variables, CAM_variables, fab_repetitions,
                                      post_process_variables, post_process_repetitions)

This experiment will require fabricating 64 unique objects.
Users will perform 3200 interactions.
9600 total measurements will be recorded.


# 2. CAD

In [2]:
from fedt_laser import build_geometry

# draw a circle in svg
def geometry_function(draw, d, radius=30):
    d.append(draw.Circle(-40, -10, radius,
                fill='none', stroke_width=1, stroke='red'))

def label_function(draw, d, label_string="L0"):
    d.append(draw.Text(x=20, y=-10, fill='blue', text=label_string, font_size=10))

# 3. CAM

In [3]:
from fedt_laser import prep_cam, do_cam

prep_cam(CAM_variables)

please open Visicut and Options > Import Settings > fedt_generated.vcsettings


/Users/vwn277/miniforge3/envs/jupyfab/lib/python3.11/zipfile.py:1549: UserWarning: Duplicate name: 'profiles/fedt_generated_20240312-153819f.xml'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/vwn277/miniforge3/envs/jupyfab/lib/python3.11/zipfile.py:1549: UserWarning: Duplicate name: 'laserprofiles/Epilog_32_Helix/Acrylic/3.0mm/fedt_generated_20240312-153819f.xml'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


# 4. Fabrication


In [4]:
from fedt_laser import prep_all_for_fab, fabricate

CAM_paths = prep_all_for_fab(vars_to_labels, geometry_function, label_function)

for CAM_path in CAM_paths:
    fabricate(CAM_path)

./
expt_L32.svg
./
expt_L33.svg
./
expt_L34.svg
./
expt_L35.svg
./
expt_L36.svg
./
expt_L37.svg
./
expt_L38.svg
./
expt_L39.svg
./
expt_L40.svg
./
expt_L41.svg
./
expt_L42.svg
./
expt_L43.svg
./
expt_L44.svg
./
expt_L45.svg
./
expt_L46.svg
./
expt_L47.svg
./
expt_L48.svg
./
expt_L49.svg
./
expt_L50.svg
./
expt_L51.svg
./
expt_L52.svg
./
expt_L53.svg
./
expt_L54.svg
./
expt_L55.svg
./
expt_L56.svg
./
expt_L57.svg
./
expt_L58.svg
./
expt_L59.svg
./
expt_L60.svg
./
expt_L61.svg
./
expt_L62.svg
./
expt_L63.svg


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

# 5. Post-Processing

In [2]:
# TODO: I was thinking that I need to print out the things to do here, but I actually
# need to attach them somehow to the LABELS in the fabrication step. so that they know what to do to which.
# and then we need to print... something here... so the user knows what to do...

# 6. Create measurement framework

In [4]:
from fedt_measure import create_experiment_csv
experiment_csv = create_experiment_csv(vars_to_labels, interaction_variables, measurement_variables, measurement_repetitions)

print("now please fill " + experiment_csv + " with data")

NameError: name 'experiment_csv' is not defined

# 6. User Interaction

In [3]:
# TODO : figure out how to deal with this. is it true that we might need to generate the csv first? I... would think so.

# 8. Tea-based hypothesis testing

In [ ]:
from fedt_tea import do_experiment

tea_results = do_experiment(measurement_variables,
                            tea_hypothesis,
                            experiment_csv,
                            CAD_variables,
                            CAM_variables,
                            post_process_variables,
                            interaction_variables)

# 9. Reporting

In [ ]:
from fedt_report import latex

latex(tea_results,
      tea_hypothesis,
      CAD_variables,
      CAM_variables,
      print_repetitions,
      post_process_variables,
      post_process_repetitions,
      interaction_variables,
      measurement_variables,
      measurement_repetitions)